<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i combine Fridata datset with SSI dataset & GPC dataset.**
*   *Frist combine FRIDA dataset with intrested columns*
*   *Second combine GPC dataset with CONCITO dataset with the help of Brickcode/GPC_level4_code*
*   *Then try to extract SSI dataset only the name and match with FRIDA dataset*
*   *Then introduce combined CONCITO_GOPC dataset to the Combined FRIODA_SSI dataset*

In [1]:
# Necessary libaries
import pandas as pd
import os

import warnings
warnings.filterwarnings("ignore")

# FRIDA Food Composition Database
***FRIDA Food Composition Database  contain mainly Nutrition value of the each food items***

*   ***Category list:*** https://frida.fooddata.dk/food/lists/grouped?lang=en
*   ***Data display inspiration:*** https://foodb.ca/foods/FOOD00011
*   ***Display on FRIDA:*** https://frida.fooddata.dk/food/754?lang=en

***Import different dataset in FRIDA***

In [2]:
# Import FRIDA Nutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Import FRIDA Group data with Group code
path_frida_data_groups="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Import FRIDA AdditionalLink data
path_frida_aditional_link="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaAdditionalLinkData.xlsx"

***Open and combine FRIDA dataset in one dataframe***

In [105]:
# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=None, header=None, skiprows=2).round(decimals = 3).iloc[:, :26]

# Lets define and Assign columns name that defined just above
column_names= ['FoodID', 'FødevareGruppe', 'FødevareNavn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 
               'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']
data_frida.columns= column_names

# lets see frist 3 and last 3 rows od the dataset
data_frida.head(3).append(data_frida.tail(3))

,FoodID,FødevareGruppe,FødevareNavn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
0,1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,...,0.8,0,0.4,10,90,3.33,0,40,0,0
1,2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
2,3,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
1183,1728,Fedtrig frugt og frø,Chiafrø,iv,1837,447,iv,3.425,21.4,21.4,...,0.887,iv,4.4,93.9,6.1,0,iv,0,0,0
1184,1729,Kornsorter,Perlespelt,iv,1386,329,iv,1.94,12.1,12.1,...,0.598,iv,1.5,88.8,11.2,0,iv,0,0,0
1185,1730,Kornsorter,Perlebyg,iv,1413,335,iv,1.455,9.1,9.1,...,0.548,iv,1.3,89.2,10.8,0,iv,0,0,0


In [4]:
# Lets open data as pandas dataframe with adjusted column name
data_frida_groups= pd.read_excel(path_frida_data_groups, sheet_name=3, index_col=None, header=0, skiprows=0)[["FoodGroupID",	"GroupNameDK",	"GroupNameUK", "ParantFoodGroupId"]]
# lets have a look row with  "Banan, rå" to have understanding of the rows
data_frida_groups[data_frida_groups.ParantFoodGroupId == 47]

,FoodGroupID,GroupNameDK,GroupNameUK,ParantFoodGroupId
39,48,Anden frugt,Other fruit,47
40,49,Kernefrugt,Pome fruit,47
41,50,Stenfrugt,Stone fruit,47
42,51,Bærfrugt,Soft fruit,47
43,52,Sydfrugt,Tropical or subtropical fruit,47
44,53,Fedtrig frugt og frø,High-fat fruit and seeds,47
45,54,Frugtsaft og juice,Fruit juices,47
46,55,Frugtprodukter,Fruit products,47


In [5]:
# Lets open with little modificaton so easy to read and understand
data_frida_aditional_link= pd.read_excel(path_frida_aditional_link, sheet_name=3, index_col=None, header=0, skiprows=0)

# lets see first 3 and last 3 rows od the dataset
data_frida_aditional_link.head(3).append(data_frida_aditional_link.tail(3))

,FødevareNavn,FoodName,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup
0,"Jordbær, rå","Strawberry, raw",1,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51,Bærfrugt,Soft fruit
1,"Æble, uspec., råt","Apple, raw, all varieties",2,Malus domestica Borkh.,A01DJ#F02.A067D$F26.A07XD,49,Kernefrugt,Pome fruit
2,"Banan, rå","Banana, raw",3,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52,Sydfrugt,Tropical or subtropical fruit
1246,"Stykker, med mycoprotein","Pieces, with mycoprotein",1793,NaN,NaN,169,Plantebaserede produkter,Plant-based products
1247,"Pølse, med sojaprotein","Sausage, with soy protein",1794,NaN,NaN,169,Plantebaserede produkter,Plant-based products
1248,"Oksekød, filet, marmoreret, rå","Beef, entrecote/rib eye, raw",1795,Bos taurus,NaN,58,Oksekød,Beef


***Combine dataset now***

In [6]:
# Frida Additionallink dataset & Nutrient Dataset
combine_fridanutrient_additionallink_dataset= data_frida_aditional_link.merge(data_frida, on= ['FoodID', 'FødevareNavn', 'FødevareGruppe'], how= 'left')
combine_fridanutrient_additionallink_dataset

# also combine frida group dataset
combine_fridanutrient_additionallink_dataset.merge(data_frida_groups[['FoodGroupID', 'ParantFoodGroupId']], on= 'FoodGroupID', how= 'left')


,FødevareNavn,FoodName,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup,Svind_%,Energy_kj,...,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg,ParantFoodGroupId
0,"Jordbær, rå","Strawberry, raw",1,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51,Bærfrugt,Soft fruit,4,162,...,0,0.4,10,90,3.33,0,40,0,0,47.0
1,"Æble, uspec., råt","Apple, raw, all varieties",2,Malus domestica Borkh.,A01DJ#F02.A067D$F26.A07XD,49,Kernefrugt,Pome fruit,10,233,...,0,0.3,15.1,84.9,2.08,0,25,0,0,47.0
2,"Banan, rå","Banana, raw",3,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52,Sydfrugt,Tropical or subtropical fruit,41,396,...,0,0.7,24.6,75.3,4.42,0,53,0,0,47.0
3,"Kartoffel, uspec., rå","Potato, raw",4,Solanum tuberosum L.,A00ZT#F02.A067V,39,Rod- og knoldgrøntsager,Root and tuber vegetables,25,326,...,0,0.9,20.5,79.5,0.833,0,10,0,0,37.0
4,"Hindbær, rå","Raspberry, raw",5,Rubus idaeus L.,A01EP#F02.A067D,51,Bærfrugt,Soft fruit,0,215,...,0,0.5,13.8,86.2,3.5,0,42,0,0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,"Pølse, med ærteprotein","Sausage, with pea protein",1791,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1245,"Pålæg i skiver, med æggehvide","Cold cuts, with eggwhite",1792,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1246,"Stykker, med mycoprotein","Pieces, with mycoprotein",1793,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1247,"Pølse, med sojaprotein","Sausage, with soy protein",1794,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
combine_fridanutrient_additionallink_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1249 entries, 0 to 1248
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   FødevareNavn                 1249 non-null   object
 1   FoodName                     1249 non-null   object
 2   FoodID                       1249 non-null   int64 
 3   TaxonomicName                620 non-null    object
 4   FoodEx2Code                  911 non-null    object
 5   FoodGroupID                  1249 non-null   int64 
 6   FødevareGruppe               1249 non-null   object
 7   FoodGroup                    1249 non-null   object
 8   Svind_%                      643 non-null    object
 9   Energy_kj                    643 non-null    object
 10  Energy_kcal                  643 non-null    object
 11  Nitrogen_til_protein_faktor  643 non-null    object
 12  Nitrogen_total_g             643 non-null    object
 13  Protein_videnskabelign_g     643 

#CONCITO dataset and GPC dataset

***Import dataset***

In [14]:
# Climate dataset
path_climate_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/ClimateData.xlsx"

#TradeSync GPC dataset
path_tradesync_excel= "https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/GS1_GPC_Dataset.xlsx" 

***CONCITO climate dataset***

In [15]:
# Loading CONCITO climate dataset into the data-frame
col_names= ["id", "Product_dk", "Category_dk", "Product_en", "Category_en", "Unit", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq_perkg", "Energy_KJ", "Fat_g", "Carb_g", "Protein_g", "Data_Source", "Comments", "GPC_Level4_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_Category_en", "GPC_Category_dk", "GPC_Level4_dk", "product_type", "GPC_level1", "Food_group", "GPC_level2", "Un/Processed", "GPC_Level3", "Extra_category"]
cols_to_use= ['Product_dk',	"Product_en", 'Category_dk', "Category_en", "GPC_Level4_dk", "Fat_g", "Carb_g", "Protein_g", "Energy_KJ", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq_perkg"]

# Original columns name are replaced with ralavant names (col_names) and open only selected columns
data_climate_selected= pd.read_excel(path_climate_data, sheet_name=1, index_col=False, header=None, skiprows=1, names= col_names).loc[:, cols_to_use]
data_climate_selected.head(5)

,Product_dk,Product_en,Category_dk,Category_en,GPC_Level4_dk,Fat_g,Carb_g,Protein_g,Energy_KJ,Agriculture,iLUC,Processing,Packaging,Transport,Retail,Total_CO2_eq_perkg
0,"Peberfrugt, rød, rå","Pepper, sweet, red, raw",Grøntsager,Vegetables,10006191,0.1,5.2,0.9,133.0,0.251618,0.033544,0.000000e+00,0.140624,0.582215,0.009921,1.017923
1,"Tomat, uspec., rå","Tomato, ripe, raw, origin unknown",Grøntsager,Vegetables,10006165,0.1,3.2,0.8,87.0,0.066407,0.008081,0.000000e+00,0.140624,0.476213,0.009921,0.701246
2,"Squash, rå","Squash, raw",Grøntsager,Vegetables,10006355,0.2,2.4,1.4,81.0,0.141884,0.040487,0.000000e+00,0.064813,0.577676,0.009921,0.834780
3,"Aubergine, rå","Aubergine, raw",Grøntsager,Vegetables,10006128,0.1,3.1,0.9,83.0,0.158010,0.019844,0.000000e+00,0.140624,0.646277,0.009921,0.974676
4,"Tomat, soltørret","Tomato, dried",Grøntsager,Vegetables,10000272,3.0,43.5,14.1,1187.0,0.905904,0.091830,-5.551115e-17,0.260522,0.677568,0.009921,1.945744


***GS1 TradeSync GPC dataset***

In [16]:
# Selective columns names
columns = ['SegmentCode', 'SegmentTitle', 'FamilyCode', 'FamilyTitle', 'ClassCode', 'ClassTitle', 'BrickCode', 'BrickTitle', 'AttributeCode', 'AttributeTitle', 'AttributeValueCode', 'AttributeValueTitle']

# Lets open data as they are
data_tradesync_excel= pd.read_excel(path_tradesync_excel, sheet_name=0, index_col=None, header=0, skiprows=0)
data_tradesync_excel.head(2).append(data_tradesync_excel.tail(2))

# Open the dataset with selective columns names
data_tradesync_selected_columns = data_tradesync_excel[columns]
data_tradesync_selected_columns.head(2).append(data_tradesync_selected_columns.tail(2))


,SegmentCode,SegmentTitle,FamilyCode,FamilyTitle,ClassCode,ClassTitle,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle
0,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001686,Airbrushes (Powered),NaN,NaN,NaN,NaN
1,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001688,Airbrushing Equipment - Replacement Parts/Acce...,20001349.0,Type of Airbrushing Equipment - Replacement Pa...,30008542.0,AIRBRUSH CONTROL VALVE
183206,77000000,Vehicle,77030000,Cars and Motorcycles,77030200,Motorcycles/ All Terrain Vehicles/ Utility Tas...,10006773,Utility Task Vehicles (UTVs)/ Recreational Off...,20002877.0,Type of Energy,30002518.0,UNIDENTIFIED
183207,77000000,Vehicle,77050000,Specialty Vehicles,77050100,Specialty Vehicles,10008050,Specialty Vehicles,NaN,NaN,NaN,NaN


In [17]:
#data_tradesync_selected_columns[data_tradesync_selected_columns.BrickCode== 10006191]	
#data_climate_selected[data_climate_selected.GPC_Level4_dk== 10006191]	

***Now, combine CONCITO_Climate & GS1_GPC_Tradesync datasets***

In [133]:
# CONCITO dataset with GPC dataset
combine_gpc_climate_dataset= data_tradesync_selected_columns.merge(data_climate_selected, left_on= 'BrickCode', right_on= 'GPC_Level4_dk', how= 'left')
combine_gpc_climate_dataset.head(2).append(combine_gpc_climate_dataset.tail(2))
#combine_gpc_climate_dataset.info()


,SegmentCode,SegmentTitle,FamilyCode,FamilyTitle,ClassCode,ClassTitle,BrickCode,BrickTitle,AttributeCode,AttributeTitle,...,Carb_g,Protein_g,Energy_KJ,Agriculture,iLUC,Processing,Packaging,Transport,Retail,Total_CO2_eq_perkg
0,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001686,Airbrushes (Powered),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70000000,Arts/Crafts/Needlework,70010000,Arts/Crafts/Needlework Supplies,70010200,Airbrushing Supplies,10001688,Airbrushing Equipment - Replacement Parts/Acce...,20001349.0,Type of Airbrushing Equipment - Replacement Pa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227356,77000000,Vehicle,77030000,Cars and Motorcycles,77030200,Motorcycles/ All Terrain Vehicles/ Utility Tas...,10006773,Utility Task Vehicles (UTVs)/ Recreational Off...,20002877.0,Type of Energy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227357,77000000,Vehicle,77050000,Specialty Vehicles,77050100,Specialty Vehicles,10008050,Specialty Vehicles,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***Exploring connecting column and feacture for FRIDA and GPC_Concito dataset***

In [150]:
combine_gpc_climate_dataset[combine_gpc_climate_dataset.Product_dk == "Jordbær, rå"].iloc[:, [6,7,8,9,10,11,12,13,16,27]]

,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle,Product_dk,Product_en,GPC_Level4_dk,Total_CO2_eq_perkg
66614,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014610.0,AFGHANISTAN,"Jordbær, rå","Strawberry, raw",10005921,0.561566
66615,10005921,Strawberries,20000743.0,Country/Zone of Origin,30016053.0,ÅLAND ISLANDS,"Jordbær, rå","Strawberry, raw",10005921,0.561566
66616,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014612.0,ALBANIA,"Jordbær, rå","Strawberry, raw",10005921,0.561566
66617,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014613.0,ALGERIA,"Jordbær, rå","Strawberry, raw",10005921,0.561566
66618,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014614.0,AMERICAN SAMOA,"Jordbær, rå","Strawberry, raw",10005921,0.561566
...,...,...,...,...,...,...,...,...,...,...
66896,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014956.0,"""EXTRA"" CLASS","Jordbær, rå","Strawberry, raw",10005921,0.561566
66897,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014608.0,CLASS I,"Jordbær, rå","Strawberry, raw",10005921,0.561566
66898,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014609.0,CLASS II,"Jordbær, rå","Strawberry, raw",10005921,0.561566
66899,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30002515.0,UNCLASSIFIED,"Jordbær, rå","Strawberry, raw",10005921,0.561566


In [153]:
combine_fridanutrient_additionallink_dataset #[combine_fridanutrient_additionallink_dataset.FødevareNavn == "Jordbær, rå"].info()

,FødevareNavn,FoodName,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup,Svind_%,Energy_kj,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
0,"Jordbær, rå","Strawberry, raw",1,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51,Bærfrugt,Soft fruit,4,162,...,0.8,0,0.4,10,90,3.33,0,40,0,0
1,"Æble, uspec., råt","Apple, raw, all varieties",2,Malus domestica Borkh.,A01DJ#F02.A067D$F26.A07XD,49,Kernefrugt,Pome fruit,10,233,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
2,"Banan, rå","Banana, raw",3,Musa x paradisiaca L.,A01LC#F01.A064L$F02.A067D,52,Sydfrugt,Tropical or subtropical fruit,41,396,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
3,"Kartoffel, uspec., rå","Potato, raw",4,Solanum tuberosum L.,A00ZT#F02.A067V,39,Rod- og knoldgrøntsager,Root and tuber vegetables,25,326,...,0.8,0,0.9,20.5,79.5,0.833,0,10,0,0
4,"Hindbær, rå","Raspberry, raw",5,Rubus idaeus L.,A01EP#F02.A067D,51,Bærfrugt,Soft fruit,0,215,...,0.8,0,0.5,13.8,86.2,3.5,0,42,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,"Pølse, med ærteprotein","Sausage, with pea protein",1791,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1245,"Pålæg i skiver, med æggehvide","Cold cuts, with eggwhite",1792,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1246,"Stykker, med mycoprotein","Pieces, with mycoprotein",1793,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1247,"Pølse, med sojaprotein","Sausage, with soy protein",1794,NaN,NaN,169,Plantebaserede produkter,Plant-based products,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Combine Frida (Combined) and gs1GPC-climate dataset
*   ***Product_dk* from combined GS1_GPC dataset. This dataset have multiple datapoint for selected string from FRIDA**
*   ***FødevareNavn* from frida dataset- This dataset have one 1 datapoint for selected string**


So it need to be merged FRIDA dataset to GPC-Concito dataset. 
After inspecting dataset:
*   Single "BrickCode" contain multiple rows with multiple AttributeCode and AttributeValue
*   It will make sense if we pass single food item with BrickCode and AttributeValueCode. because there are multiple BrickCode but single AttributeValueCode

In [158]:
# explore seleted columns that seems relavant
combine_fida_gpc_concito_dataset= combine_gpc_climate_dataset.merge(combine_fridanutrient_additionallink_dataset, left_on= 'Product_dk', right_on= 'FødevareNavn', how= 'left')
combine_fida_gpc_concito_dataset.head(2).append(combine_fida_gpc_concito_dataset.tail(2)).iloc[:, [6,7,8,9,10,11,12,13,16,27,28,29,30,31,32,33,34,35]]

,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle,Product_dk,Product_en,GPC_Level4_dk,Total_CO2_eq_perkg,FødevareNavn,FoodName,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup
0,10001686,Airbrushes (Powered),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001688,Airbrushing Equipment - Replacement Parts/Acce...,20001349.0,Type of Airbrushing Equipment - Replacement Pa...,30008542.0,AIRBRUSH CONTROL VALVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227356,10006773,Utility Task Vehicles (UTVs)/ Recreational Off...,20002877.0,Type of Energy,30002518.0,UNIDENTIFIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227357,10008050,Specialty Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
# lets see the row with "Jordbær, rå"
combine_fida_gpc_concito_dataset[combine_fida_gpc_concito_dataset.FødevareNavn == "Jordbær, rå"].iloc[:, [6,7,8,9,10,11,12,13,16,27,28,29,30,31,32,33,34,35]]

,BrickCode,BrickTitle,AttributeCode,AttributeTitle,AttributeValueCode,AttributeValueTitle,Product_dk,Product_en,GPC_Level4_dk,Total_CO2_eq_perkg,FødevareNavn,FoodName,FoodID,TaxonomicName,FoodEx2Code,FoodGroupID,FødevareGruppe,FoodGroup
66614,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014610.0,AFGHANISTAN,"Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit
66615,10005921,Strawberries,20000743.0,Country/Zone of Origin,30016053.0,ÅLAND ISLANDS,"Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit
66616,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014612.0,ALBANIA,"Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit
66617,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014613.0,ALGERIA,"Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit
66618,10005921,Strawberries,20000743.0,Country/Zone of Origin,30014614.0,AMERICAN SAMOA,"Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66896,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014956.0,"""EXTRA"" CLASS","Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit
66897,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014608.0,CLASS I,"Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit
66898,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30014609.0,CLASS II,"Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit
66899,10005921,Strawberries,20002737.0,Quality (UNECE Standard),30002515.0,UNCLASSIFIED,"Jordbær, rå","Strawberry, raw",10005921,0.561566,"Jordbær, rå","Strawberry, raw",1.0,Fragaria x ananassa Duchesne,A01EA#F02.A067D,51.0,Bærfrugt,Soft fruit


In [ ]:
from google.colab import files
combine_fida_gpc_concito_dataset.to_excel('combine_fida_gpc_concito_dataset.xlsx', encoding = 'utf-8-sig') 
#files.download('combine_fida_gpc_concito_dataset.xlsx')

# Data by Frederik

***Columns description***

*   **itemname:** name provided by the merchant. It often, but not always, contains a enough information to ID the product.  Sometimes weight, or other information is also provided here.
*   **itemnumber:** the product itemnumber. It can be a standardized GTIN 13 number ( 12-13 digits) supported by GS1 or a non standard number created by the retailer to identify i.e. Bananas
*   **category:** a category code not sure how to interpret and often missing
*   **percent_purchases:** Percent off all receipt lines in the dataset where each line is a purchas

In [37]:
# Import data
path_ssi_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/cpd_summary.csv"

# Open data as csv file
data_ssi= pd.read_csv(path_ssi_data, sep= ";", index_col=False, encoding='latin-1')

# lets see frist 3 and last 3 rows od the dataset
data_ssi.head(3).append(data_ssi.tail(5))

# only some of the rows of different kinds of the banana
data_ssi.iloc[[11, 12, 14, 18], :]  

,itemname,itemnumber,category,percent_purchases
11,"ØKO BANANER, LØSE",5712873336944,89,"0,567"
12,"BANANER,LØSE",5711044628659,NaN,"0,565"
14,BANANER,406,0000000030,"0,419"
18,"BANANER,LØSE",5711044628659,89,"0,266"


In [57]:
# lets explore the SSI dataset with GTIN code from GS1
gtin_list= ['05707402008577', '05707402006641', '05707402005231', '05707402007990', '05707402008119', '05707402008171', '00000057014388'] # ['15705001150090', '05705001150086']

for gtin in gtin_list:
  print(data_ssi[data_ssi.itemnumber == gtin[1:]])
  print("*"*80)


Empty DataFrame
Columns: [itemname, itemnumber, category, percent_purchases]
Index: []
********************************************************************************
          itemname     itemnumber    category percent_purchases
112525  ØLGAVEÆSKE  5707402006641  0000000020                 0
********************************************************************************
              itemname     itemnumber    category percent_purchases
37248  BOSTON AMERICAN  5707402005231  0000000020                 0
37249  BOSTON PALE ALE  5707402005231          84                 0
********************************************************************************
                itemname     itemnumber    category percent_purchases
54394      GIN N´BEER 7%  5707402007990  0000000020                 0
102672  THISTED GIN BEER  5707402007990          84                 0
********************************************************************************
                 itemname     itemnumber catego

#Combine SSI data with Combined Frida-GS1GPC-climate dataset
*   **"itemname" from SSI dataset and "FødevareNavn" from Combined Frida-GPC-CONCITO dataset**
*    **Merging or Joining whole dataset do not making sense here, so i pick one row from SSI dataset and match with other dataset based on best match with Record Link library**

# TradeSync GTIN Link data from GS1 

In [33]:
import glob
import xml.etree.ElementTree as ET

#Import gs1 xmll files
path_tradesync_data= "/content/drive/MyDrive/Data_gs1_xml/"
xmls = glob.glob(path_tradesync_data + '*.xml')
print("Toal amount of xml files: ", len(xmls))

Toal amount of xml files:  1060


In [34]:
# Funccction tot xtract data from xml files
def gs1_xml_reader(file):
  """This function will take xml file eand return GTIN and Brick code as key:value pair"""
  tree = ET.parse(file)
  root = tree.getroot()
  keys_of_items_to_find= ("gtin", "gpcCategoryCode", "brandName", 'functionalName', 'descriptionShort', "additionalTradeItemIdentification", "uniformResourceIdentifier", "ingredientStatement")
  
  #Lets start the dictionary to collecct items 
  dit= dict([(key,[]) for key in keys_of_items_to_find])

  # Loop over the tags that make sense (from list above)
  for list_item in keys_of_items_to_find:
    # Again  looping to get the text value
    
    for item in root.iter(list_item):
      if item.text not in dit[list_item]:
        dit[list_item].append(item.text)
      #else:
        #dit[list_item].append(0)

  # Lets see what we have collected from the loop
  return dit


gs1_xml_reader(xmls[0])

#df_xml= pd.DataFrame.from_dict(d, orient='index').transpose()
#df_xml

{'gtin': ['05000387905498', '05000387905634'],
 'gpcCategoryCode': ['10000263'],
 'brandName': ['Bells'],
 'functionalName': ['Whisky'],
 'descriptionShort': ['Original'],
 'additionalTradeItemIdentification': ['624860'],
 'uniformResourceIdentifier': ['https://gdsnprodwebstorage.blob.core.windows.net/gdsnprodwebfileblob/5790000000722/05000387905498/208/ReferencedFileDetailInformationModule/05000387905498T.1.png',
  'https://gdsnprodwebstorage.blob.core.windows.net/gdsnprodwebfileblob/5790000000722/05000387905498/208/ReferencedFileDetailInformationModule/05000387905498T.2.png',
  'https://gdsnprodwebstorage.blob.core.windows.net/gdsnprodwebfileblob/5790000000722/05000387905498/208/ReferencedFileDetailInformationModule/05000387905498T.3.png',
  'https://gdsnprodwebstorage.blob.core.windows.net/gdsnprodwebfileblob/5790000000722/05000387905634/208/ReferencedFileDetailInformationModule/05000387905634.1.png',
  'https://gdsnprodwebstorage.blob.core.windows.net/gdsnprodwebfileblob/5790000000

In [64]:
# Lets loop over all the files and extract the GTIN code and associated Brick-codes
keys_of_items_to_find= ("gtin", "gpcCategoryCode", "brandName", 'functionalName', 'descriptionShort', "additionalTradeItemIdentification", "uniformResourceIdentifier", "ingredientStatement")
dit= dict([(key,[0]) for key in keys_of_items_to_find])
df_with_gtin_list = pd.DataFrame()
for xml in xmls[:]:
  df_with_gtin_list= df_with_gtin_list.append(gs1_xml_reader(xml), ignore_index = True)
  
df_with_gtin_list

,gtin,gpcCategoryCode,brandName,functionalName,descriptionShort,additionalTradeItemIdentification,uniformResourceIdentifier,ingredientStatement
0,"[05000387905498, 05000387905634]",[10000263],[Bells],[Whisky],[Original],[624860],[https://gdsnprodwebstorage.blob.core.windows....,[Spiritus]
1,"[04008239649805, 04008239655257, 04008239251091]",[10000522],[Vitakraft],"[Daily food, Hauptfutter]","[Life Marsvin, LIFE Power, 600g, MS, Life Powe...",[25109],[],[]
2,[06942138946529],[10005155],[Impulse],[Dykkerbriller],[Hydro-Swim Dykkerbriller 3-asst],[I-22057],[],[]
3,"[05701018044677, 05701018044646]",[10000055],[Bähncke],"[Bähncke Chili Bearnais SR 325g-8stk, Chili Be...","[Bähncke Chili Bearnais SR 325g-8stk, Chili Be...","[154528, 5701018044646]",[http://nrgdesign.dk/orkla.php?id=72662],"[Ingredienser: Vand/vatten, rapsolie/-olja, mo..."
4,"[28711295755952, 18711295755955, 08711295755958]",[99999999],[Maj],[HALSKÆDE LÆDERSNØRRE M/VEDHÆNG],[HALSKÆDE LÆDERSNØRRE M/VEDHÆNG],[6159],[],[]
...,...,...,...,...,...,...,...,...
1055,"[08420197061271, 08420197001277]",[10000028],[Øvrige],[Dessertost],[MANCHEGO D.O. 200 G. 6 MDR.],[6791],[http://lactalis.dk/admin/public/getimage.ashx...,[Ingredienser: Pasteuriseret fåreMÆLK. salt. M...
1056,[05764630930254],[10000379],[hårpynt],[Hårpynt],[hårpynt],[993025],[],[]
1057,"[15704080955589, 05704080955155, 0570408095516...",[10005840],"[3-Stjernet , 3-Stjernet - Den originale, 3-St...","[3-STJERNET SALGSDISPLAY (120 STK.), SPEGEPØLS...","[3-STJERNET SALGSDISPLAY (120 STK.), SPEGEPØLS...","[911840, 911131, 911132, 911133, 911134, 911135]",[https://gdsnprodwebstorage.blob.core.windows....,[Ingredienser: Til 100 g færdigvare er anvendt...
1058,[05410522494518],[10000045],[Bensdorp],[Cocoa powder],[20/22 DT UTZ],[100317-UTZ-793],[],"[Ingredienser: kakaopulver, Ingredients: cocoa..."


In [72]:
from google.colab import files
df_with_gtin_list= df_with_gtin_list[["gtin", "gpcCategoryCode", "brandName", 'functionalName', 'descriptionShort', 'additionalTradeItemIdentification', "uniformResourceIdentifier","ingredientStatement"]].iloc[:,:]
df_with_gtin_list.to_excel('df_with_gtin_list.xlsx', encoding = 'utf-8-sig') 
#files.download('df_with_gtin_list.xlsx')

#Combine SSI data and GTIN:BRICK-CODE dataset
*   ***FødevareNavn* from frida dataset- This dataset have one 1 datapoint for selected string**
*   ***BrickTitle* from GS1_GPC dataset. This sdataset have multiple datapoint for selected string from FRIDA.FødevareNavn**

So it need to be merged FRIDA dataset to GPC-Concito dataset. 

#Lets create the pipeline to extract data from the combination of ISS dataset, Frida dataset and GS1 GPC-dataset

In [ ]:
!pip3 install eansearch
from eansearch import EANSearch

ean_list= ['05701018044677', '05000387905634', 25109, 5712873336944, 5712580050171, 20008932, 20014018, 20023393, 5712580000114, "05766632526800"]
ean = ean_list[0]
apiToken = "a09851e102099d04448394e0b838bc345c8fd98a"


"""eansearch = EANSearch(apiToken)
eanList = eansearch.productSearch('Bananas')
for product in eanList:
	print(product["ean"], " is ", product["name"].encode("utf-8"))"""
 
lookup = EANSearch(apiToken)
name = lookup.barcodeLookup(ean)
print(ean, " is ", name)